# Assess Task Compute Time and Data Sizes
To get a better idea how they are each different and provide quantitative descriptions in the paper

In [1]:
from pathlib import Path
import pandas as pd
import json

Configuration

In [2]:
run_dir = Path('prod-runs/baseline_retrain-20/')

## Load in Each Type of Task
We'll need the runtimes and method sizes

In [3]:
tasks = []
for path in run_dir.glob('*-results.json'):
    records = pd.read_json(path, lines=True)
    records['total_size'] = records['message_sizes'].apply(lambda x: sum(x.values())) / 1e6
    tasks.append(records[['method', 'time_running', 'total_size', 'message_sizes']])
tasks = pd.concat(tasks)

## Group by Mean
Print out the statistics

In [4]:
tasks.groupby('method').describe(percentiles=[0.5])

time_running                                                  \
                      count        mean         std         min         50%   
method                                                                        
evaluate               24.0    1.193537    0.080184    1.076494    1.167433   
run_calculator        209.0  367.582606  312.969203    8.625638  299.034220   
run_sampling          144.0   11.614806    8.853649    1.169105   10.293323   
train                  80.0  258.351150    7.254764  245.876419  258.610408   

                            total_size                                  \
                        max      count       mean       std        min   
method                                                                   
evaluate           1.358199       24.0   9.730499  0.276886   9.365479   
run_calculator  1075.854432      211.0   0.019898  0.007965   0.005874   
run_sampling      36.362183      144.0   3.306343  0.181427   3.103596   
train            272.921305       80.0  20.830953  0.549740  19.986104   

                                      
                      50%        max  
method                                
evaluate         9.811633  10.014385  
run_calculator   0.021278   0.031866  
run_sampling     3.275466   3.981202  
train           20.807840  21.725822

## Get the size of the inference task
How many structures per chunk

In [5]:
with (run_dir / 'runparams.json').open() as fp:
    config = json.load(fp)

In [6]:
config['infer_chunk_size']

500